In [1]:
from bs4 import BeautifulSoup
import csv
import numpy as np
import scraperwiki
import urllib2
import urllib
import time

In [2]:
url = 'http://www.ufc.com/rankings?fb_comment_id=6086517324953#f391bfa61a74bf2'
html = scraperwiki.scrape(url)
soup = BeautifulSoup(html, 'lxml')

In [3]:
page = urllib2.urlopen(url)
soup = BeautifulSoup(page.read(), 'lxml')
# print soup

In [4]:
tables = soup.find_all('div', {'class': 'ranking-list tall'})
tables.extend(soup.find_all('div', {'class': 'ranking-list tall center-row'}))

In [ ]:
fighters = []
for table in tables:
    wcname = str(table.find('div', {'class': 'weight-class-name'}).contents[0]).strip()
    if 'Pound-for-Pound' in wcname:
        continue
    else:
#         print table.a.contents[0]
        fighters.append(table.a.contents[0])
        other_ranked = table.find_all('td', {'class': 'name-column'})
        for ranked in other_ranked:
            print ranked.a.contents[0]
        

In [ ]:
fighters

In [ ]:
names = []
for table in tables:
    all_td = table.find_all('a')
    all_td_str = set([x.string for x in all_td if x.string is not None])
    all_td_str = [x.encode('utf-8').strip() for x in all_td_str if 'County' not in x]
    all_td_str = [x for x in all_td_str if 'District' not in x and 'spade' not in x and 'Piercer' not in x and '\xbc' not in x and 'Generals' not in x and 'Marquis' not in x]
    all_td_str = [x for x in all_td_str if 'home' not in x and 'Yama' not in x]
    all_td_str = [x for x in all_td_str if 'Spade' not in x]
    all_td_str = [x for x in all_td_str if ' ' in x]
    all_td_str = [[x] for x in all_td_str if len(x.split(' ')) < 4]
    names += all_td_str

In [ ]:
with open('./chinese_names.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(names)

In [ ]:
for year in range(1880, 1930):
    post_params = {
    'top' : '100',
    'year' : str(year)
    }
    post_args = urllib.urlencode(post_params)
    urllib2.urlopen(url, post_args)
    headers = {
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Language' : 'en-US,en;q=0.5',
        'Connection' : 'keep-alive',
        'Host' : 'www.ssa.gov',
        'Referer' : 'http://www.ssa.gov/cgi-bin/popularnames.cgi',
        'User-Agent' : 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:21.0) Gecko/20100101 Firefox/21.0'
        }

    # With POST data:
    page = urllib.urlopen(url, post_args, headers)
    soup = BeautifulSoup(page.read(), 'lxml')
    rank = soup.find('table', {'border': '1', 'bordercolor' : '#aaaabb'})
    all_td = rank.find_all('td')
    all_td_str = [x.string for x in all_td]
    all_td_str.pop(-1)
    B = np.reshape(all_td_str, (-1, 3))
    B = [list(x)[1:] for x in B]
    
    with open('./names/'+str(year)+'_names.csv', 'w') as f:
        w = csv.writer(f)
        w.writerows(B)